In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from multiprocessing import Pool, cpu_count

import numpy as np
import matplotlib.pyplot as plt

from utils import read_arr_help, split_ds

In [ ]:
def save_as_npz(
    data_path: str, data_size: int, seed: int = 42, test_size: float = 0.2
) -> None:
    """Read and save .dat data in a .npz file. The data retrieved are 
    the array of speckle (both real and fourier), the x axis and the output values.
    
    TODO: Use an input to specify the names of files to be retrieved. 

    Args:
        data_path (str): Path to the files.
        data_size (int): Size of a single array in the data.
        seed (int, optional): Seed to retrieve pseudo-randomly training and test datasets. Defaults to 42.
        test_size (float, optional): Size (in %) of the test set. Defaults to 0.2.
    """
    paths = []
    for file in os.listdir(data_path):
        path = os.path.join(data_path, file)
        if file[:4] == "eval":
            # energy value is a scalar
            paths.append((path, 1, 1))
        elif file[:8] == "speckleF" or file[:8]=="densityF":
            # speckleF has real and imag part
            paths.append((path, data_size, (1, 2)))
        else:
            # valid for speckleR, just real
            paths.append((path, data_size, 1))
    print(paths)
    # append extra vector with x axis
    extra_paths = []
    for path in paths:
        filename = os.path.basename(path[0])[:-4]
        if filename == "speckleR":
            extra_paths.append((path[0], data_size, 0, "x_axis"))
        elif filename == "speckleF":
            extra_paths.append((path[0], data_size, 0, "csi_axis"))
    print(extra_paths)

    cpu = np.minimum(len(paths), cpu_count() // 2)
    p = Pool(cpu)

    # data are in the same files, so to avoid concurrent accesses the loading is split
    data = list(p.imap(read_arr_help, paths))
    data.extend(list(p.imap(read_arr_help, extra_paths)))

    results = split_ds(data, seed=seed, test_size=test_size)

    for key in results:
        outname = key + "_" + os.path.basename(data_path)
        print("\nSaving {0} dataset as {1}".format(key, outname))
        np.savez(str(outname) + ".npz", **{el[1][:]: el[0] for el in results[key]})
    return

In [ ]:
#save_as_npz("../data/data_L28", 512)

In [ ]:
data = np.load("dataset/train_data_L28.npz")
data.files

In [ ]:
speckle = data['speckleR']

ro = data['densityprofile']
psi = np.sqrt(ro)

x_ax = data['x_axis']

energy = data['evalues']

print(speckle.shape, ro.shape, x_ax.shape, energy.shape)

In [ ]:
x = np.linspace(0, 28, 512)
x[1] - x[0]

In [ ]:
dx = x[1] - x[0]
grad_psi = np.gradient(psi, x, axis=1)

pred_energy = np.sum(grad_psi**2 * dx, axis=1) + np.sum(ro * speckle * dx, axis=1)

print(pred_energy.shape, dx)

In [ ]:
print(pred_energy[:5], energy[:5], energy[:5]/pred_energy[:5])

In [ ]:
fig = plt.figure(figsize=(12,12))
plt.scatter(pred_energy, energy)

In [ ]:
grad_ro = np.gradient(ro, x, axis=1)

pred_energy2 = np.sum(grad_ro**2 * dx / ro, axis=1) / 8 + np.sum(ro * speckle * dx, axis=1)

In [ ]:
test_data = np.load("dataset/test_data_L28.npz")

speckleF = data['speckleF']
speckleF_test = test_data['speckleF']

In [ ]:
real_speckle = np.concatenate((np.real(speckleF[:,1:29]), np.real(speckleF_test[:,1:29])), axis=None) 
print(real_speckle.mean())
imag_speckle = np.concatenate((np.imag(speckleF[:,1:29]), np.imag(speckleF_test[:,1:29])), axis=None)
print(imag_speckle.mean())


speckle_tot = np.append(real_speckle, imag_speckle)

In [ ]:
print(np.mean(speckle_tot), np.std(speckle_tot))

In [ ]:
speckleF[0,29]

In [ ]:
from torch import nn

In [ ]:
a = nn.ReLU()